In [52]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 17 15:17:00 2023

@author: gunahn

@contributer: Evan 
"""

import pandas as pd
import math
import glob
import os 

from tqdm import tqdm

#imports file and renames the column headers
def import_VF_csv(VFname):
    VF = pd.read_csv(VFname)
    VF.columns = ['Location', 'Tailx', 'Taily', 'TailLikely', 'BaseTailx', 'BaseTaily', 'BaseTailLikely', 'CBodyx', 'CBodyy', 'CBodyLikely', 'neckx', 'necky' , 'neckLikely' , 'Snoutx', 'Snouty', 'SnoutLikely', 'RFrx', 'RFry', 'RFrLikely', 'LFrx', 'LFry', 'LFrLikely', 'RBackx', 'RBacky', 'RBackLikely', 'LBackx', 'LBacky', 'LBackLikely']
    return(VF)


#cleans up the location data so that there are no outliers
def clean_VF(VF):
    newVF = VF.copy()
    changes = 0
    errordist = 0
    for n in range(2, len(newVF.index), 1):
        if n == 2:
            pastx, pasty = newVF.loc[n].at['Tailx'], newVF.loc[n].at['Taily']
        else:
            if errordist > 6:
                errordist = 0
            elif math.dist([float(newVF.loc[n].at['Tailx']), float(newVF.loc[n].at['Taily'])],[float(newVF.loc[n].at['BaseTailx']), float(newVF.loc[n].at['BaseTaily'])]) > (8 * math.dist([float(newVF.loc[n].at['Tailx']), float(newVF.loc[n].at['Taily'])],[float(newVF.loc[n].at['Snoutx']), float(newVF.loc[n].at['Snouty'])])):
                newVF.loc[n, 'Tailx'] = pastx
                newVF.loc[n, 'Taily'] = pasty
                changes += 1
                errordist += 1
            #checks if the tail location has moved more than the max speed. If so, it will take the old location of the tail, only if the old location is in a reasonable range of the length of the tail
            elif math.dist([float(pastx), float(pasty)], [float(newVF.loc[n].at['Tailx']), float(newVF.loc[n].at['Taily'])]) > 150:
                if math.dist([float(pastx), float(pasty)],[float(newVF.loc[n].at['BaseTailx']), float(newVF.loc[n].at['BaseTaily'])]) < 250*1.5:
                    newVF.loc[n, 'Tailx'] = pastx
                    newVF.loc[n, 'Taily'] = pasty
                    changes += 1
                    errordist += 1
            elif math.dist([float(newVF.loc[n].at['Tailx']), float(newVF.loc[n].at['Taily'])],[float(newVF.loc[n].at['BaseTailx']), float(newVF.loc[n].at['BaseTaily'])]) > 250*1.2: #This checks if distance to base of tail is longer than length of tail
                newVF.loc[n, 'Tailx'] = pastx
                newVF.loc[n, 'Taily'] = pasty
                changes += 1
                errordist += 1
            else:
                errordist = 0
            pastx, pasty = newVF.loc[n].at['Tailx'], newVF.loc[n].at['Taily']
    print('Total number of changes: ' + str(changes))
    return(newVF)


def add_kinematics(df):
    '''
    This function will take a dataframe (df) and will compute kinematic data such as speed...
    Will return the modified df as the output.
    '''
    speeds = [None] * len(df.index)
    
    df["Speed"] = speeds
    
    for n in range(4, len(df.index), 1): #this does not compute the kinematic data for the first frame since there is no previous frame to get speed from
        df.loc[n, 'Speed'] = math.dist([float(df.loc[n, 'CBodyx']), float(df.loc[n, 'CBodyy'])], [float(df.loc[n-1, 'CBodyx']), float(df.loc[n-1, 'CBodyy'])]) * 30

    return df
    

def combine_dfs(video_df, ttl_df):
    '''
    This function will take the video dataframe and the ttl dataframe and add them togehter
    '''
    video_df.drop([0,1], axis=0, inplace= True) # This removes the first two rows from the video df since it does not contain useful data
    # reset index of video_df
    video_df.reset_index(drop=True, inplace=True)

    ttl_df_new = ttl_df.iloc[:len(video_df.index)]
    result = pd.concat((video_df, ttl_df_new), axis = 1)
    return result

def categorize_ttl_data(df):
    category = [None] * len(df.index)
    turned = False
    for n in range(len(df.index)):
        if df.loc[n, 'Trial On']:
            if df.loc[n, 'In Turn Area']:
                turned = True
            if turned:
                category[n] = "Turned"
            else:
                category[n] = "Trial on"
        else:
            turned = False
            
    df["Category"] = category
    return df




In [53]:
def VF_ttl_combiner(VFname,ttlname,animal_number):
    #VFname = 'Day_20_2ch_GFAP_D_2022-11-29T10_02_16DLC_resnet50_maze_221230Dec30shuffle1_250000.csv'
    #ttlname = 'Day_20_2ch_GFAP_D_2022-11-29T10_02_15.csv'

    VF = import_VF_csv(VFname) #coordinate data to be cleaned
    newVF = clean_VF(VF) # This df contains the coordinate data

    newVF_kinematic = add_kinematics(newVF) # this is the positional dataframe with speed added on as a column
    ttl_action_df = pd.read_csv(ttlname) # this df contains the action data for the mouse
    ttl_action_df.columns = ["Total Entries", "Rewarded Choice", "Frames", "Timestamp", "Entries(even if out of task)", "left entry", "right entry", "Trial On", "In Turn Area", "Left correct+rewarded", "Right correct+rewarded", "Initiation(start) counter", "Left correct+omission", "Right correct+omission", "Left error", "Right error", "Airpuff", "Left or Right correct(Left=True)", "L/R switch count"]
    categorized_ttl = categorize_ttl_data(ttl_action_df)


    #########combination section

    output = combine_dfs(newVF_kinematic, categorized_ttl)
    
    try:
        os.makedirs('/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/GFAP_Behavior_0418/{}'.format(animal_number))
    except FileExistsError:
        # directory already exists
        pass
    for_file_name = ttlname.split('/')[-1]
    print(animal_number)
    print(for_file_name)
    output.to_csv('/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/GFAP_Behavior_0418/{}/{}_{}'.format(animal_number,animal_number, for_file_name))

In [5]:
path = '/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/2chSw_221020_Bonsai/GFAP_221021_DLC'

In [13]:
all_list =  glob.glob("/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/2chSw_221020_Bonsai/GFAP_221021_DLC/*.csv")


In [47]:
VFname_list = []
ttlname_list = []
for count, value in enumerate(all_list):
    if count%3 == 0:
        ttlname_list.append(value)
    elif count%3 == 2:
        VFname_list.append(value)

'Day_14_2ch_GFAP_D_2022-11-15T10_56_41.csv'

In [56]:
for idx in tqdm(range(len(VFname_list))): 
    animal_number = idx%5 +1 
    
    VF_ttl_combiner(VFname_list[idx], ttlname_list[idx],animal_number )
    
    

  0%|          | 0/70 [00:00<?, ?it/s]/var/folders/zx/mr5vw08n18jfshsf89hf76v40000gn/T/ipykernel_20106/1805611762.py:20: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  VF = pd.read_csv(VFname)


Total number of changes: 1964
1
Day_14_2ch_GFAP_D_2022-11-15T10_56_41.csv


  1%|▏         | 1/70 [00:17<19:48, 17.22s/it]/var/folders/zx/mr5vw08n18jfshsf89hf76v40000gn/T/ipykernel_20106/1805611762.py:20: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  VF = pd.read_csv(VFname)


Total number of changes: 3272
2
Day_14_2ch_GFAP_D_2022-11-15T11_22_54.csv


  3%|▎         | 2/70 [00:35<20:09, 17.79s/it]/var/folders/zx/mr5vw08n18jfshsf89hf76v40000gn/T/ipykernel_20106/1805611762.py:20: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  VF = pd.read_csv(VFname)


...

 97%|█████████▋| 68/70 [21:39<00:43, 21.52s/it]/var/folders/zx/mr5vw08n18jfshsf89hf76v40000gn/T/ipykernel_20106/1805611762.py:20: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  VF = pd.read_csv(VFname)


Total number of changes: 2396
4
Day_27_2ch_GFAP_D_2022-12-14T11_47_10.csv


 99%|█████████▊| 69/70 [21:55<00:20, 20.07s/it]/var/folders/zx/mr5vw08n18jfshsf89hf76v40000gn/T/ipykernel_20106/1805611762.py:20: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  VF = pd.read_csv(VFname)


Total number of changes: 3513
5
Day_27_2ch_GFAP_D_2022-12-14T12_11_26.csv


100%|██████████| 70/70 [22:14<00:00, 19.06s/it]


In [26]:
VFname_list = []
ttlname_list = []
for count, value in enumerate(all_list):
    if count%3 == 0:
        ttlname_list.append(value)
    elif count%3 == 2:
        VFname_list.append(value)